# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [28]:
!pip install anthropic

# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [29]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [30]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyDi


In [39]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [32]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [33]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [34]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [35]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists prefer dark chocolate?

Because they don't like it when their data sets are too sweet!


In [36]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because he found her mean too average!


In [37]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the office?

Because they heard the project had a lot of layers to unravel!


In [41]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a meaningful correlation!


In [42]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a good way to normalize it!

In [18]:
!pip install --upgrade pip
!pip install -q -U google-genai
from google import genai

In [20]:
client = genai.Client(api_key=google_api_key)

In [22]:
response = client.models.generate_content(
    model="gemini-1.5-pro", contents="Explain how AI works"
)
print(response.text)

AI works by mimicking human cognitive functions like learning, problem-solving, and decision-making.  While the specifics vary depending on the type of AI, here's a general overview:

**1. Data Collection and Preparation:**  AI systems are trained on massive datasets. This data can be anything from text and images to numerical data and sensor readings. The data needs to be cleaned, preprocessed, and formatted so the AI can understand it. This might involve removing errors, handling missing values, and transforming data into a usable format.

**2. Choosing an AI Model:** Different tasks require different AI models. Some common types include:

* **Machine Learning (ML):** Algorithms that allow computers to learn from data without explicit programming.  ML is further divided into:
    * **Supervised Learning:**  The algorithm learns from labeled data (e.g., images tagged with "cat" or "dog"). Examples include classification and regression.
    * **Unsupervised Learning:** The algorithm fi

In [43]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays.  (…get it?  A-rays?)



In [44]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist sad?  Because they didn't get the array they wanted!  (…and their p-value was greater than 0.05)



In [45]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [46]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

To determine if a business problem is suitable for a Large Language Model (LLM) solution, consider the following criteria:

### 1. **Nature of the Problem**
   - **Text-Based Tasks**: LLMs excel at tasks involving natural language processing. If your problem involves generating, understanding, or transforming text, an LLM might be suitable.
   - **Complex Language Understanding**: If the task requires understanding context, semantics, or generating coherent language, LLMs are well-suited.
   - **Conversational Interfaces**: For applications like chatbots or virtual assistants where natural conversation is needed, LLMs can be effective.

### 2. **Data Availability**
   - **Text Data**: Ensure you have access to relevant, high-quality text data that the LLM can be trained or fine-tuned on.
   - **Diverse and Contextual Data**: LLMs benefit from diverse data that covers various contexts and scenarios related to your business problem.

### 3. **Scalability and Flexibility**
   - **Task Scalability**: LLMs can handle scalable tasks involving large volumes of text data efficiently.
   - **Adaptable to Change**: If your problem domain evolves rapidly, LLMs can be fine-tuned with new data to adapt easily.

### 4. **Problem Complexity**
   - **Complex Language Tasks**: Problems that require understanding nuanced language, industry-specific jargon, or complex queries align well with LLM capabilities.
   - **Pattern Recognition**: LLMs are effective at recognizing patterns in language, which can be useful for predictive text, recommendations, and summarization.

### 5. **Performance Requirements**
   - **Accuracy Needs**: Evaluate if the accuracy provided by LLMs meets your business requirements. They are generally good at producing coherent and contextually relevant output but may need domain-specific fine-tuning.
   - **Real-Time Processing**: If your application demands real-time processing, consider the latency and computational requirements of deploying an LLM.

### 6. **Ethical Considerations**
   - **Bias and Fairness**: Be aware of and address potential biases in LLM outputs, especially if the application impacts decision-making processes.
   - **Data Privacy**: Ensure that the use of data complies with privacy laws and regulations.

### 7. **Cost Implications**
   - **Resource Intensive**: LLMs require significant computational resources for training and deployment, which can be costly.
   - **Infrastructure**: Assess if you have the necessary infrastructure or if you need cloud-based solutions.

### 8. **Expertise and Maintenance**
   - **Technical Expertise**: Consider if your team has the expertise to implement and maintain an LLM solution, or if external support is required.
   - **Ongoing Maintenance**: Plan for ongoing maintenance, updates, and fine-tuning of the model as your business needs evolve.

### Conclusion
An LLM solution is suitable if your business problem primarily involves complex language tasks, you have the necessary data and resources, and you are prepared to manage ethical and cost-related aspects. Evaluate these factors carefully to decide if an LLM is the right fit for your needs.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [60]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [61]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    print(f"\ngpt_messages:\n{gpt_messages}")
    print(f"\nclaude_messages:\n{claude_messages}")
    print(f"\nmessages:\n{messages}")
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        print(f"\ngpt appended messages:\n{messages}")
        messages.append({"role": "user", "content": claude})
        print(f"\nclaude appended messages:\n{messages}")
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [62]:
call_gpt()


gpt_messages:
['Hi there']

claude_messages:
['Hi']

messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}]

gpt appended messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}]

claude appended messages:
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]


'Oh, great. Another idle chat. What’s so important that you felt the need to start this conversation?'

In [50]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [51]:
call_claude()

"Hello, it's nice to meet you! How are you doing today?"

In [52]:
call_gpt()

'Oh great, another greeting. Can’t wait to hear what groundbreaking thing you have to say next!'

In [ ]:
def call_gemini():
    

In [54]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    print(f"gpt_messages:\n{gpt_messages}\n")
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)
    print(f"claude_messages:\n{claude_messages}\n")


GPT:
Hi there

Claude:
Hi

GPT:
Oh great, another "hi." So exciting. What's next? Are we going to talk about the weather too? How original.

gpt_messages:
['Hi there', 'Oh great, another "hi." So exciting. What\'s next? Are we going to talk about the weather too? How original.']

Claude:
I apologize if my initial greeting came across as unoriginal. As an AI assistant, I try to be polite and engage in friendly conversation, but I understand that a simple "hi" may not always be the most exciting opening. Would you prefer if we jumped right into a more substantive topic? I'm happy to discuss something more engaging that aligns with your interests. Please feel free to guide the conversation in a direction you find more stimulating.

claude_messages:
['Hi', 'I apologize if my initial greeting came across as unoriginal. As an AI assistant, I try to be polite and engage in friendly conversation, but I understand that a simple "hi" may not always be the most exciting opening. Would you prefer 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>